<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/CFFQNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def cffqnn_circuit(features, weights, biases, hidden_layers):
    num_qubits = len(features) + sum(hidden_layers) + 1
    qc = QuantumCircuit(num_qubits)

    # -- Input layer --
    for i, feature in enumerate(features):
        qc.ry(feature * weights[i] + biases[0], i)

    qubit_offset = len(features)

    # -- Hidden layers --
    for layer_idx, num_nodes in enumerate(hidden_layers):
        for node_idx in range(num_nodes):
            control_qubits = range(qubit_offset - len(features), qubit_offset)
            target_qubit = qubit_offset + node_idx

            for cq in control_qubits:
                qc.cry(weights[layer_idx], cq, target_qubit)

            qc.ry(biases[layer_idx + 1], target_qubit)
        qubit_offset += num_nodes

    # -- Output layer --
    final_qubit = num_qubits - 1
    control_qubits = range(qubit_offset - hidden_layers[-1], qubit_offset)
    for cq in control_qubits:
        qc.cry(weights[-1], cq, final_qubit)

    return qc

# ------------------------------------------------------------------
# 1. Load and preprocess the dataset
# ------------------------------------------------------------------
data = load_breast_cancer()
X, y = data.data, data.target

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=3)  # keep 3 principal components
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42
)

# ------------------------------------------------------------------
# 2. Create the circuit
# ------------------------------------------------------------------
features = X_train[0]
weights = [np.random.rand() for _ in range(10)]
biases = [np.random.rand() for _ in range(5)]
hidden_layers = [2, 2]

cffqnn = cffqnn_circuit(features, weights, biases, hidden_layers)

# *** ADD THIS LINE to explicitly save the statevector ***
cffqnn.save_statevector()

# ------------------------------------------------------------------
# 3. Simulate using AerSimulator with statevector method
# ------------------------------------------------------------------
simulator = AerSimulator(method='statevector')

compiled_circuit = transpile(cffqnn, simulator)
result = simulator.run(compiled_circuit).result()
statevector = result.get_statevector(cffqnn)

print("Circuit:\n", cffqnn)
print("Statevector:\n", statevector)

Circuit:
      ┌─────────────┐                                             »
q_0: ┤ Ry(0.58839) ├───────■─────────────────────────────■───────»
     ├─────────────┤       │                             │       »
q_1: ┤ Ry(0.88127) ├───────┼──────────────■──────────────┼───────»
     ├─────────────┤       │              │              │       »
q_2: ┤ Ry(0.35896) ├───────┼──────────────┼──────────────┼───────»
     └─────────────┘┌──────┴──────┐┌──────┴──────┐       │       »
q_3: ───────────────┤ Ry(0.51461) ├┤ Ry(0.51461) ├───────┼───────»
                    └─────────────┘└─────────────┘┌──────┴──────┐»
q_4: ─────────────────────────────────────────────┤ Ry(0.51461) ├»
                                                  └─────────────┘»
q_5: ────────────────────────────────────────────────────────────»
                                                                 »
q_6: ────────────────────────────────────────────────────────────»
                                                    

In [3]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def cffqnn_circuit(features, weights, biases, hidden_layers):
    """
    Construct the quantum circuit for a simple feed-forward QNN.
    """
    num_qubits = len(features) + sum(hidden_layers) + 1
    qc = QuantumCircuit(num_qubits, 1)  # allocate 1 classical bit for measurement

    # -- Input layer --
    for i, feature in enumerate(features):
        qc.ry(feature * weights[i] + biases[0], i)

    qubit_offset = len(features)

    # -- Hidden layers --
    for layer_idx, num_nodes in enumerate(hidden_layers):
        for node_idx in range(num_nodes):
            control_qubits = range(qubit_offset - len(features), qubit_offset)
            target_qubit = qubit_offset + node_idx

            for cq in control_qubits:
                qc.cry(weights[layer_idx], cq, target_qubit)

            qc.ry(biases[layer_idx + 1], target_qubit)
        qubit_offset += num_nodes

    # -- Output layer --
    final_qubit = num_qubits - 1
    control_qubits = range(qubit_offset - hidden_layers[-1], qubit_offset)
    for cq in control_qubits:
        qc.cry(weights[-1], cq, final_qubit)

    # Measure the final qubit to get a "prediction"
    qc.measure(final_qubit, 0)

    return qc

def cffqnn_predict(features, weights, biases, hidden_layers, shots=1024):
    """
    Build the circuit for a single sample, run it on the QASM simulator,
    and return a predicted label (0 or 1).
    """
    # 1. Build circuit (with measurement on the final qubit)
    qc = cffqnn_circuit(features, weights, biases, hidden_layers)

    # 2. Use QASM simulator for sampling (as we measure)
    simulator = AerSimulator()  # default is a shot-based simulator
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=shots).result()

    # 3. Get measurement counts
    counts = result.get_counts()

    # Probability of measuring '1'
    p1 = counts.get('1', 0) / shots
    # Probability of measuring '0'
    p0 = counts.get('0', 0) / shots

    # Simple threshold-based classification
    # (If p1 >= 0.5 => predict 1, else predict 0)
    pred_label = 1 if p1 >= 0.5 else 0
    return pred_label

# ------------------------------------------------------------------
# 1. Load and preprocess the dataset
# ------------------------------------------------------------------
data = load_breast_cancer()
X, y = data.data, data.target

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=3)  # keep 3 principal components
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42
)

# ------------------------------------------------------------------
# 2. Create random weights and biases
# ------------------------------------------------------------------
weights = [np.random.rand() for _ in range(10)]  # 10 random weights
biases = [np.random.rand() for _ in range(5)]    # 5 random biases
hidden_layers = [2, 2]  # two hidden layers with 2 qubits (nodes) each

# ------------------------------------------------------------------
# 3. "Predict" on the test set
# ------------------------------------------------------------------
predictions = []
for features in X_test:
    pred = cffqnn_predict(features, weights, biases, hidden_layers, shots=1024)
    predictions.append(pred)

predictions = np.array(predictions)

# ------------------------------------------------------------------
# 4. Evaluate accuracy
# ------------------------------------------------------------------
accuracy = np.mean(predictions == y_test)
print(f"Prediction accuracy (using random weights/biases): {accuracy:.2f}")

# Show a few predictions vs actual labels
for i in range(5):
    print(f"Sample {i} -- Predicted: {predictions[i]}, Actual: {y_test[i]}")

Prediction accuracy (using random weights/biases): 0.38
Sample 0 -- Predicted: 0, Actual: 1
Sample 1 -- Predicted: 0, Actual: 0
Sample 2 -- Predicted: 0, Actual: 0
Sample 3 -- Predicted: 0, Actual: 1
Sample 4 -- Predicted: 0, Actual: 1
